In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

import pandas as pd

StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 3, Finished, Available)

In [2]:
df=pd.read_parquet('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/raw_data/uber_raw_data.parquet')

df.head()

StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 4, Finished, Available)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2023-01-02 23:15:03,2023-01-02 23:38:03,NaN,15.80,NaN,None,132,137,0,70.0,0.0,0.5,8.18,7.80,1.0,89.98,NaN,NaN
1,2,2023-01-01 08:15:18,2023-01-01 08:48:41,5.0,20.04,2.0,N,132,142,1,70.0,0.0,0.5,10.00,6.54,1.0,91.79,2.5,1.25
2,2,2023-01-01 15:47:29,2023-01-01 16:56:39,4.0,18.11,2.0,N,132,48,1,70.0,0.0,0.5,10.00,13.10,1.0,98.35,2.5,1.25
3,2,2023-01-01 17:46:46,2023-01-01 18:21:38,4.0,21.21,2.0,N,132,243,1,70.0,0.0,0.5,17.05,13.75,1.0,103.55,0.0,1.25
4,2,2023-01-01 23:48:55,2023-01-02 00:22:05,4.0,19.49,2.0,N,132,231,1,70.0,0.0,0.5,9.00,6.77,1.0,91.02,2.5,1.25


In [3]:

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 5, Finished, Available)

In [4]:
#1 Create Dim_Datetime Dimension from Original Dataframe df

Dim_Datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 6, Finished, Available)

In [5]:
Dim_Datetime['tpep_pickup_datetime'] 	= Dim_Datetime['tpep_pickup_datetime']
Dim_Datetime['pick_hour']  				= Dim_Datetime['tpep_pickup_datetime'].dt.hour
Dim_Datetime['pick_day'] 				= Dim_Datetime['tpep_pickup_datetime'].dt.day
Dim_Datetime['pick_month'] 				= Dim_Datetime['tpep_pickup_datetime'].dt.month
Dim_Datetime['pick_year'] 				= Dim_Datetime['tpep_pickup_datetime'].dt.year
Dim_Datetime['pick_weekday'] 			= Dim_Datetime['tpep_pickup_datetime'].dt.weekday

Dim_Datetime['tpep_dropoff_datetime'] 	= Dim_Datetime['tpep_dropoff_datetime']
Dim_Datetime['drop_hour'] 				= Dim_Datetime['tpep_dropoff_datetime'].dt.hour
Dim_Datetime['drop_day'] 				= Dim_Datetime['tpep_dropoff_datetime'].dt.day
Dim_Datetime['drop_month'] 				= Dim_Datetime['tpep_dropoff_datetime'].dt.month
Dim_Datetime['drop_year'] 				= Dim_Datetime['tpep_dropoff_datetime'].dt.year
Dim_Datetime['drop_weekday'] 			= Dim_Datetime['tpep_dropoff_datetime'].dt.weekday


Dim_Datetime['datetime_id']				 = Dim_Datetime.index


Dim_Datetime = Dim_Datetime[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]



StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 7, Finished, Available)

In [6]:
#2 Create Dim_Passenger_Count
Dim_Passenger_Count 						= df[['passenger_count']].reset_index(drop=True)
Dim_Passenger_Count['passenger_count_id'] 	= Dim_Passenger_Count.index
Dim_Passenger_Count 						= Dim_Passenger_Count[['passenger_count_id','passenger_count']]


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 8, Finished, Available)

In [7]:
#3 Create Dim_Trip_Distance

Dim_Trip_Distance 						= df[['trip_distance']].reset_index(drop=True)
Dim_Trip_Distance['trip_distance_id'] 	= Dim_Trip_Distance.index
Dim_Trip_Distance 						= Dim_Trip_Distance[['trip_distance_id','trip_distance']]


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 9, Finished, Available)

In [8]:
#4 Create Dim_Rate_Code

rate_code_type = {
                    1:"Standard rate",
                    2:"JFK",
                    3:"Newark",
                    4:"Nassau or Westchester",
                    5:"Negotiated fare",
                    6:"Group ride"
                }

Dim_Rate_Code                    = df[['RatecodeID']].reset_index(drop=True)
Dim_Rate_Code['rate_code_id']    = Dim_Rate_Code.index
Dim_Rate_Code['rate_code_name']  = Dim_Rate_Code['RatecodeID'].map(rate_code_type)
Dim_Rate_Code                    = Dim_Rate_Code[['rate_code_id','RatecodeID','rate_code_name']]


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 10, Finished, Available)

In [12]:
#5 Create Dim_Payment_Type

payment_type_name = {
                        1:"Credit card",
                        2:"Cash",
                        3:"No charge",
                        4:"Dispute",
                        5:"Unknown",
                        6:"Voided trip"
                    }
					
Dim_Payment_Type 						= df[['payment_type']].reset_index(drop=True)
Dim_Payment_Type['payment_type_id'] 	= Dim_Payment_Type.index
Dim_Payment_Type['payment_type_name'] 	= Dim_Payment_Type['payment_type'].map(payment_type_name)
Dim_Payment_Type 						= Dim_Payment_Type[['payment_type_id','payment_type','payment_type_name']]


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 14, Finished, Available)

In [13]:
#6 Create Fact Table

Fact_Table = df.merge(Dim_Passenger_Count, left_on='trip_id', right_on='passenger_count_id') \
             .merge(Dim_Trip_Distance, left_on='trip_id', right_on='trip_distance_id') \
             .merge(Dim_Rate_Code, left_on='trip_id', right_on='rate_code_id') \
             .merge(Dim_Datetime, left_on='trip_id', right_on='datetime_id') \
             .merge(Dim_Payment_Type, left_on='trip_id', right_on='payment_type_id') \
             [[ 'trip_id','VendorID', 'datetime_id', 'passenger_count_id','trip_distance_id', 
                'rate_code_id', 'store_and_fwd_flag','payment_type_id', 'fare_amount', 'extra', 
                'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]

#Fact_Table.head()

StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 15, Finished, Available)

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,None,0,70.0,0.0,0.5,8.18,7.80,1.0,89.98
1,1,2,1,1,1,1,N,1,70.0,0.0,0.5,10.00,6.54,1.0,91.79
2,2,2,2,2,2,2,N,2,70.0,0.0,0.5,10.00,13.10,1.0,98.35
3,3,2,3,3,3,3,N,3,70.0,0.0,0.5,17.05,13.75,1.0,103.55
4,4,2,4,4,4,4,N,4,70.0,0.0,0.5,9.00,6.77,1.0,91.02


In [ ]:
# Dim_Datetime
# Dim_Passenger_Count
# Dim_Trip_Distance
# Dim_Rate_Code
# Dim_Payment_Type
# Fact_Table


In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,BooleanType
from pyspark.sql.functions import col, lit, expr, concat, concat_ws, sha2,round,concat_ws,regexp_replace


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 17, Finished, Available)

In [18]:
## Converting Pandas dataframe to Spark Dataframe

sparkDF_Dim_Datetime=spark.createDataFrame(Dim_Datetime) 
sparkDF_Dim_Passenger_Count=spark.createDataFrame(Dim_Passenger_Count) 
sparkDF_Dim_Trip_Distance=spark.createDataFrame(Dim_Trip_Distance) 
sparkDF_Dim_Rate_Code=spark.createDataFrame(Dim_Rate_Code) 
sparkDF_Dim_Payment_Type=spark.createDataFrame(Dim_Payment_Type) 
sparkDF_Fact_Table=spark.createDataFrame(Fact_Table) 


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 20, Finished, Available)

In [21]:
# Write Spark Dataframes into Delta format into processed folders

sparkDF_Dim_Datetime.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/dim_datetime')
sparkDF_Dim_Passenger_Count.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/dim_passenger_count')
sparkDF_Dim_Trip_Distance.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/dim_trip_distance')
sparkDF_Dim_Rate_Code.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/dim_rate_code')
sparkDF_Dim_Payment_Type.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/dim_payment_type')
sparkDF_Fact_Table.write.format("delta").mode("overwrite").save('abfss://8e8726e4-c8fe-4ed5-b972-9bc6737bce7f@onelake.dfs.fabric.microsoft.com/4fa89bca-d81f-4864-b4a9-66e67711cb9c/Files/processed/fact_table')


# sparkDF_Dim_Passenger_Count.write.format("delta").mode("overwrite").save('processed/dim_passenger_count')
# sparkDF_Dim_Trip_Distance.write.format("delta").mode("overwrite").save('processed/dim_trip_distance')
# sparkDF_Dim_Rate_Code.write.format("delta").mode("overwrite").save('processed/dim_rate_code')
# sparkDF_Dim_Payment_Type.write.format("delta").mode("overwrite").save('processed/dim_payment_type')
# sparkDF_Fact_Table.write.format("delta").mode("overwrite").save('processed/fact_table')



StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 23, Finished, Available)

In [24]:
## Command to check the database names

##%%sql
###show DATABASES;

# db name: uber_lakehouse


StatementMeta(, 5a11e2e7-4420-4979-b666-5612888aa548, 26, Finished, Available)

<Spark SQL result set with 2 rows and 1 fields>